In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import pickle
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import normalize
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier

In [2]:
df_raw = pd.read_csv("../data/train.csv").drop("id",axis=1)
df_test_raw = pd.read_csv("../data/test.csv").drop("id",axis=1)

In [4]:
df_raw.describe(percentiles=[0.99])

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_84,feat_85,feat_86,feat_87,feat_88,feat_89,feat_90,feat_91,feat_92,feat_93
count,61878.00000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,...,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000,61878.000000
mean,0.38668,0.263066,0.901467,0.779081,0.071043,0.025696,0.193704,0.662433,1.011296,0.263906,...,0.070752,0.532306,1.128576,0.393549,0.874915,0.457772,0.812421,0.264941,0.380119,0.126135
std,1.52533,1.252073,2.934818,2.788005,0.438902,0.215333,1.030102,2.255770,3.474822,1.083340,...,1.151460,1.900438,2.681554,1.575455,2.115466,1.527385,4.597804,2.045646,0.982385,1.201720
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
99%,6.00000,5.000000,15.000000,12.000000,1.000000,1.000000,4.000000,9.000000,18.000000,5.000000,...,1.000000,8.000000,12.000000,5.000000,10.000000,6.000000,21.000000,4.000000,5.000000,2.000000
max,61.00000,51.000000,64.000000,70.000000,19.000000,10.000000,38.000000,76.000000,43.000000,30.000000,...,76.000000,55.000000,65.000000,67.000000,30.000000,61.000000,130.000000,52.000000,19.000000,87.000000


In [5]:
# Récupération des labels
labels = df_raw["target"].values
y = []
for label in labels:
    y.append(int(label[-1:])-1)
y = np.asarray(y)
# Récupérations des features sous forme de ndarray
df_raw = df_raw.drop("target", axis=1)
X = df_raw.values
# X_test = df_test_raw.values
print('X shape : '+str(X.shape))
print('y shape : '+str(y.shape))

X shape : (61878, 93)
y shape : (61878,)


In [8]:
1 - (np.count_nonzero(X) / (X.shape[0] * X.shape[1]))

0.7934165981134573

In [4]:
# Feature selection
#k_bests = np.arange(5, 90, step=5)
k_bests=[10, 50, 93]
#clf = MultinomialNB()
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

for k_best in k_bests:
    selection = SelectKBest(chi2, k=k_best)
    X_new = selection.fit_transform(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2,
                                                        random_state=42)
    
    clf.fit(X_train, y_train)
    pred = clf.predict_proba(X_test)
    print('--- k_best = '+str(k_best))
    print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))


--- k_best = 10
1.72


KeyboardInterrupt: 

In [ ]:
feature_list = [i for i in range(93)]


# Get numerical feature importances
importances = clf.feature_importances_
print(importances.shape)
print(np.where(importances > 0.01))
reduced = np.where(importances > 0.01)
print(reduced[0])


In [20]:
from sklearn.svm import SVC
# Feature selection
k_bests = [80]
#clf = MultinomialNB()
#clf = SVC(kernel='rbf')
clf_tmp = ExtraTreesClassifier(n_estimators=250, random_state=0, n_jobs=-1, verbose=True)
for k_best in k_bests:
    selection = SelectKBest(chi2, k=k_best)
    X_new = selection.fit_transform(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2,
                                                        random_state=0)
    
    clf = CalibratedClassifierCV(clf_tmp, method='isotonic', cv=5)
    clf.fit(X_train, y_train)
    print("Fit terminé")
    pred = clf.predict_proba(X_test)
    pred_for_acc = clf.predict(X_test)
    print('--- k_best = '+str(k_best))
    print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
    print(classification_report(y_test, pred_for_acc, target_names=target_names))

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   34.3s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   44.5s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    0.9s finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   40.6s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   37.7s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:   49.5s finished
[Parallel(n_jobs=4)]: Done  42 

Fit terminé


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    4.6s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    4.0s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    5.6s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    4.3s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    5.6s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    4.5s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    5.7s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    4.8s
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    5.9s finished
[Parallel(n_jobs=4)]: Done  42 tasks    

--- k_best = 80
0.49
             precision    recall  f1-score   support

    class_1       0.72      0.54      0.62       370
    class_2       0.76      0.84      0.79      3205
    class_3       0.60      0.57      0.59      1546
    class_4       0.81      0.54      0.65       566
    class_5       0.97      0.98      0.97       542
    class_6       0.94      0.94      0.94      2823
    class_7       0.73      0.70      0.71       572
    class_8       0.91      0.92      0.92      1703
    class_9       0.88      0.88      0.88      1049

avg / total       0.82      0.82      0.82     12376



[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:    3.2s finished


In [6]:
X_norm = Normalizer().fit_transform(X)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2,
random_state=42)

target_names = ['class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

In [10]:
clf = MultinomialNB()
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

CPU times: user 34.2 ms, sys: 3.32 ms, total: 37.5 ms
Wall time: 59.1 ms
CPU times: user 9.7 ms, sys: 1.8 ms, total: 11.5 ms
Wall time: 20 ms
Kaggle log loss metric : 0.82
Accuracy : 0.72
             precision    recall  f1-score   support

    class_1       0.55      0.04      0.08       399
    class_2       0.59      0.95      0.73      3178
    class_3       0.50      0.12      0.20      1561
    class_4       0.92      0.02      0.04       538
    class_5       0.93      0.93      0.93       565
    class_6       0.88      0.90      0.89      2884
    class_7       0.78      0.39      0.53       552
    class_8       0.76      0.87      0.81      1674
    class_9       0.75      0.81      0.78      1025

avg / total       0.72      0.72      0.66     12376



In [11]:
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

CPU times: user 27.3 s, sys: 435 ms, total: 27.7 s
Wall time: 22.1 s
CPU times: user 556 ms, sys: 82.2 ms, total: 638 ms
Wall time: 420 ms
0.61
Accuracy : 0.80
             precision    recall  f1-score   support

    class_1       0.79      0.37      0.50       399
    class_2       0.70      0.88      0.78      3178
    class_3       0.61      0.46      0.52      1561
    class_4       0.83      0.38      0.53       538
    class_5       0.97      0.96      0.97       565
    class_6       0.92      0.95      0.94      2884
    class_7       0.80      0.56      0.66       552
    class_8       0.87      0.93      0.90      1674
    class_9       0.85      0.88      0.87      1025

avg / total       0.80      0.80      0.79     12376



In [17]:
clf = ExtraTreesClassifier(n_estimators=250, min_samples_split=2, random_state=0, n_jobs=-1,
                           criterion='entropy')
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

CPU times: user 8.12 s, sys: 238 ms, total: 8.35 s
Wall time: 9.41 s
CPU times: user 260 ms, sys: 60 ms, total: 320 ms
Wall time: 223 ms
Kaggle log loss metric : 0.73
Accuracy : 0.80
             precision    recall  f1-score   support

    class_1       0.84      0.43      0.57       399
    class_2       0.70      0.89      0.78      3178
    class_3       0.63      0.47      0.54      1561
    class_4       0.85      0.36      0.51       538
    class_5       0.97      0.98      0.97       565
    class_6       0.92      0.95      0.94      2884
    class_7       0.77      0.54      0.64       552
    class_8       0.87      0.92      0.89      1674
    class_9       0.87      0.88      0.87      1025

avg / total       0.80      0.80      0.79     12376



In [12]:
clf = KNeighborsClassifier(n_neighbors=110, n_jobs=-1)
%time clf.fit(X_train, y_train)
%time pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("Kaggle log loss metric : %.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

CPU times: user 9.65 s, sys: 113 ms, total: 9.76 s
Wall time: 28.7 s


KeyboardInterrupt: 

Kaggle log loss metric : 0.61
Accuracy : 0.75
             precision    recall  f1-score   support

    class_1       0.55      0.34      0.42       399
    class_2       0.64      0.87      0.74      3178
    class_3       0.51      0.36      0.42      1561
    class_4       0.82      0.07      0.14       538
    class_5       0.93      0.96      0.95       565
    class_6       0.94      0.90      0.92      2884
    class_7       0.79      0.53      0.63       552
    class_8       0.84      0.89      0.86      1674
    class_9       0.76      0.85      0.80      1025

avg / total       0.75      0.75      0.73     12376



In [ ]:
clf = SVC(kernel='rbf', C=1.0 , cache_size=4000, decision_function_shape='ovo')
clf.fit(X_train, y_train)
pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print("Accuracy : %.2f" % accuracy_score(y_test, pred_for_acc))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

In [ ]:
print(pred_for_acc[:20])
print(y_test[:20])